Products:
    1.Fuel consumption reducer
    2.Risk data driver scoring based on style
from driverstyle...

 
Hi Sean,

 

As requested, these are the tables from Mix; namely

Asset Master  (Daily Stamp)
Driver Master  (Daily Stamp)
Events Library
Events  (extracted every hour and stored. The last reloads run just before midnight which extracts all the days data)
Position    (stored every 30 seconds)
Trips (extracted every hour and stored. The last reloads run just before midnight which extracts all the days data)
 
Fields that are highlighted in yellow in the attached files, are fields that are brought in for easy reference and these are the Organisation Name, Driver Name, Asset Registration and Vin Number and Events description.

 

    
Please note that for Mix, we have data only available for the current day and stored as highlighted in the red font above

 

 

 

https://stackabuse.com/hierarchical-clustering-with-python-and-scikit-learn/

In [1]:
import xlrd

In [2]:
import sys
import re
import pandas as pd
import numpy as np

In [3]:
data_path='/media/lnr-ai/christo/github_repos/logistics/data/'

In [4]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [12]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
onlyfiles

['Asset Master.xlsx',
 'Driver Master.xlsx',
 'Events Library Master.xlsx',
 'Events.xlsx',
 'Fuel Consumption & Events Data (1).xlsx',
 'Position.xlsx',
 'Trips.xlsx']

In [59]:
originalflist=['Asset Master.xlsx',
 'Driver Master.xlsx',
 'Events Library Master.xlsx',
 'Events.xlsx',
 'Fuel Consumption & Events Data (1).xlsx',
 'Position.xlsx',
 'Trips.xlsx']

In [13]:
trips_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/Trips.xlsx', sheet_name='Sheet1')

In [14]:
list(trips_df)

['GroupId',
 'Organisation Name',
 'Source',
 'DriverId',
 'Driver Name',
 'AssetId',
 'VinNumber',
 'RegistrationNumber',
 '__KEY_root',
 'DistanceKilometers',
 'DrivingTime',
 'Duration',
 'EndOdometerKilometers',
 'EndPositionId',
 'FirstDepart',
 'FuelUsedLitres',
 'LastHalt',
 'MaxAccelerationKilometersPerHourPerSecond',
 'MaxDecelerationKilometersPerHourPerSecond',
 'MaxRpm',
 'MaxSpeedKilometersPerHour',
 'PulseValue',
 'StandingTime',
 'StartOdometerKilometers',
 'StartPositionId',
 'TripEnd',
 'TripId',
 'TripStart']

In [15]:
trips_df_=trips_df[~trips_df.DistanceKilometers.isna()].copy()
trips_df_[trips_df_.DistanceKilometers>0].shape

(10214, 28)

In [16]:
len(set(trips_df_.DriverId))

734

In [ ]:
sheet_name='Data - Pipe Delimiter'

In [19]:
filename='Fuel Consumption & Events Data (1).xlsx'
sheet_name='Fuel Consumption & Events Data'
df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name=sheet_name)

In [24]:
df.iloc[3]

Unnamed: 0                                                NaN
Unnamed: 1                                                NaN
Unnamed: 2                                                NaN
Unnamed: 3                                                NaN
Unnamed: 4                                                NaN
Unnamed: 5                                                NaN
Unnamed: 6                                                NaN
Unnamed: 7                                                NaN
Unnamed: 8                                                NaN
Unnamed: 9                          .Impact Detected >13 (MV)
Unnamed: 10                         .Impact Detected >13 (MV)
Unnamed: 11          .TAR - Alert Possible Accident >20km/h/s
Unnamed: 12          .TAR - Alert Possible Accident >20km/h/s
Unnamed: 13                   .TAR - Excessive Idling > 10min
Unnamed: 14                   .TAR - Excessive Idling > 10min
Unnamed: 15    .TAR - Over Speeding > 90km/h > 1 Min (Trucks)
Unnamed:

In [17]:
for filename in onlyfiles:
    print(filename)
    df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')
    print(list(df))

Asset Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'AdditionalMobileDevice', 'AssetId', 'AssetImage', 'AssetImageUrl', 'AssetTypeId', 'Colour', 'CreatedBy', 'CreatedDate', 'CreatedTime', 'Description', 'EngineHours', 'EngineNumber', 'FleetNumber', 'FmVehicleId', 'FuelType', 'Icon', 'IconColour', 'IsConnectedTrailer', 'IsDefaultImage', 'Make', 'Model', 'Notes', 'Odometer', 'RegistrationNumber', 'TargetFuelConsumption', 'TargetFuelConsumptionUnits', 'TargetHourlyFuelConsumption', 'TargetHourlyFuelConsumptionUnits', 'UserState', 'VinNumber', 'Year']
Driver Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'Driver Name', 'DriverId', 'EmployeeNumber', 'ExtendedDriverIdType', 'FmDriverId', 'IsSystemDriver', 'MobileNumber']
Events Library Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'Event Description', 'EventType', 'EventTypeId']
Events.xlsx
['Name', 'Description', 'EventType', '__KEY_root', 'Armed', 'AssetId_u0', 'DriverId_u0', 'EventDateTime', 'EventId', 'EventLati

XLRDError: No sheet named <'Sheet1'>

### Asset Master.xlsx

Asset Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'AdditionalMobileDevice', 'AssetId', 'AssetImage', 'AssetImageUrl', 'AssetTypeId', 'Colour', 'CreatedBy', 'CreatedDate', 'CreatedTime', 'Description', 'EngineHours', 'EngineNumber', 'FleetNumber', 'FmVehicleId', 'FuelType', 'Icon', 'IconColour', 'IsConnectedTrailer', 'IsDefaultImage', 'Make', 'Model', 'Notes', 'Odometer', 'RegistrationNumber', 'TargetFuelConsumption', 'TargetFuelConsumptionUnits', 'TargetHourlyFuelConsumption', 'TargetHourlyFuelConsumptionUnits', 'UserState', 'VinNumber', 'Year']

### Driver Master.xlsx

['GroupId', 'Organisation Name', 'Source', 'Driver Name', 'DriverId', 'EmployeeNumber', 'ExtendedDriverIdType', 'FmDriverId', 'IsSystemDriver', 'MobileNumber']

### Events Library Master.xlsx

'GroupId', 'Organisation Name', 'Source', 'Event Description', 'EventType', 'EventTypeId'

### Events.xlsx

['Name', 'Description', 'EventType', '__KEY_root', 'Armed', 'AssetId_u0', 'DriverId_u0', 'EventDateTime', 'EventId', 'EventLatitude', 'EventLongitude', 'EventSpeed', 'EventTotalTime', 'EventTypeId', 'GroupId', 'OdometerKilometres_u0', 'OdometerKilometresEnd_u0', 'Priority', 'Source', 'Value']

### Position.xlsx

['GroupId', 'Organisation Name', 'Source', 'DriverId', 'Driver Name', 'AssetId', 'VinNumber', 'RegistrationNumber', 'ageOfReadingSeconds', 'altitudeMetres', 'distanceSinceReadingKilometres', 'EventDate', 'EventDateTime', 'EventHour', 'Eventtime', 'formattedAddress', 'hdop', 'heading', 'isAvl', 'latitude', 'longitude', 'numberOfSatellites', 'odometerKilometres', 'pdop', 'positionId', 'speedKilometresPerHour', 'speedLimit', 'timestamp', 'vdop']

### Trips.xlsx

['GroupId', 'Organisation Name', 'Source', 'DriverId', 'Driver Name', 'AssetId', 'VinNumber', 'RegistrationNumber', '__KEY_root', 'DistanceKilometers', 'DrivingTime', 'Duration', 'EndOdometerKilometers', 'EndPositionId', 'FirstDepart', 'FuelUsedLitres', 'LastHalt', 'MaxAccelerationKilometersPerHourPerSecond', 'MaxDecelerationKilometersPerHourPerSecond', 'MaxRpm', 'MaxSpeedKilometersPerHour', 'PulseValue', 'StandingTime', 'StartOdometerKilometers', 'StartPositionId', 'TripEnd', 'TripId', 'TripStart']

In [ ]:
filename='Position.xlsx'
position_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [ ]:
position_df.speedKilometresPerHour

In [ ]:
position_df.iloc[5005736]

In [31]:
filename='Driver Master.xlsx'
driver_master_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [32]:
set(driver_master_df.DriverId)

{nan,
 3.3305698794800824e+18,
 6.652289605037818e+18,
 8.093161458326888e+18,
 7.169467641638535e+18,
 8.744478296203829e+18,
 8.726659782798377e+18,
 nan,
 3.8059750176701153e+18,
 8.132445079506649e+18,
 8.952341289603039e+18,
 4.910083541040103e+18,
 7.405450040768668e+18,
 3.9915576974804255e+18,
 3.626017972311818e+18,
 -6.626867539558152e+17,
 6.061962310230475e+18,
 6.219844380796767e+17,
 -8.050844404945157e+18,
 -3.3719859843360676e+18,
 -4.0228169960390477e+18,
 4.7743004033295e+18,
 -1.7260087770960184e+18,
 4.266749141649475e+18,
 -4.810468960488716e+18,
 -7.442358142579735e+17,
 -3.302124514094938e+18,
 -4.6562565332600416e+17,
 -2.5164527313362938e+17,
 -4.003497403919564e+17,
 4.2713093013209764e+18,
 3.3673241668030976e+18,
 6.026843393298729e+18,
 6.396936621783976e+18,
 4.702256903919962e+18,
 1.615159226755304e+18,
 -5.903466581901932e+18,
 -6.017375675564652e+18,
 -2.5272267380059965e+18,
 -3.9492579078585354e+18,
 3.1723743285480356e+18,
 -2.7605958128016225e+18,


### Events

In [25]:
filename='Events Library Master.xlsx'
events_library_master_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [26]:
list(events_library_master_df)

['GroupId',
 'Organisation Name',
 'Source',
 'Event Description',
 'EventType',
 'EventTypeId']

In [27]:
filename='Events.xlsx'
events_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')

In [54]:
list(events_df)

['Name',
 'Description',
 'EventType',
 '__KEY_root',
 'Armed',
 'AssetId_u0',
 'DriverId_u0',
 'EventDateTime',
 'EventId',
 'EventLatitude',
 'EventLongitude',
 'EventSpeed',
 'EventTotalTime',
 'EventTypeId',
 'GroupId',
 'OdometerKilometres_u0',
 'OdometerKilometresEnd_u0',
 'Priority',
 'Source',
 'Value']

In [28]:
set(events_df.DriverId_u0)

{nan,
 nan,
 6.366881122800517e+18,
 9.108944395108016e+18,
 6.215628227223372e+18,
 8.323304274917835e+18,
 9.77250791234216e+17,
 8.998042535176602e+18,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 3.1723743285480356e+18,
 -4.0381738421844664e+18,
 1.0256276274522605e+18,
 1.0124015246410712e+18,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 1.3071701446926502e+16,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4.5632222382230477e+18,
 1.9217582021590223e+18,
 6.225612698741885e+17,
 -8.541184624809981e+17,
 1.26734095467638e+18,
 -6.824364709692627e+18,
 -4.281541024229415e+18,
 -8.629775268147745e+18,
 8.885926099329765e+17,
 1.0283813638631711e+18,
 9.660091279767101e+17,
 6.402685439517012e+18,
 -2.794218548286992e+17,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 2.2195715149128547e+17,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 -7.614076142884733e+18,
 1

In [33]:
driverset=list(set(intersection(list(events_df.DriverId_u0),list(driver_master_df.DriverId))))

### This is how many many drivers we have:

In [34]:
len(driverset)

682

In [35]:
events_df.shape

(114832, 20)

In [36]:
len(set(events_df.EventTypeId))

436

### Group driver by event

In [37]:
set(events_df.Description)

{'(Do not use) Ignition On >1km (MV)',
 '* Assistance Request',
 '* Harsh Cornering',
 '* Harsh Left Cornering',
 '* Harsh Right Cornering',
 '* Ignition Off',
 '* Ignition On',
 '* Impact Detect Moderate',
 '* Impact Detect Severe',
 '* Input 1 High',
 '* Input 1 Low',
 '* Input 2 Low',
 '* Low Battery',
 '* Low Internal Battery',
 '* Low Remote Control Battery',
 '* Low Vehicle Battery',
 '* OBC unit reset',
 '* Panic Alert',
 '* Power Disconnected',
 '* Power Reconnected',
 '*After Hour',
 '*Asset in Workshop Hours',
 '*Left Depot Late 7-10',
 '*No Go Zone',
 '.Actros Engine Brake',
 '.Actros foot brake > 50km/h',
 '.Alarm - Dash Tamper (2310I)',
 '.Alarm - Dash Tamper (Limp)',
 '.Alarm - Dash Tamper (TL)',
 '.Alarm - Panic Pressed',
 '.Alarm - Panic Pressed (MV)',
 '.Alarm - Power Down',
 '.Alert - Excessive Over Revving (>2400 RPM)',
 '.Alert - Over Speeding >100km/h (Trucks)',
 '.Alert - Over Speeding >120km/h (Bus)',
 '.Alert - Over Speeding >140km/h (LDV)',
 '.Alert - Possible 

In [38]:
behaviour=list(set(events_df.Description))

In [40]:
len(behaviour)

441

In [41]:
negative_behaviour=['*No Go Zone','* Impact Detect Moderate',
 '* Impact Detect Severe','* Harsh Cornering',
 '* Harsh Left Cornering',
 '* Harsh Right Cornering','.TAR - Over Speeding > 100km/h (Trucks)',
                                      '.Alert - Excessive Over Revving (>2400 RPM)',
 '.Alert - Over Speeding >100km/h (Trucks)',
 '.Alert - Over Speeding >120km/h (Bus)',
 '.Alert - Over Speeding >140km/h (LDV)',
 '.Alert - Possible Accident > 20km/h/s',
 '.Alert - Prohibited driving (22h00 - 02h45)',
'.Harsh Braking > 10km/h/s (MV)(LDV)',
 '.Harsh Braking > 8km/h/s (MV)(Trucks)',
 '.Harsh Braking >10km/h/s (LDV/Fuso F) (MV)',
                                      '.Over Reving (Act/Axo)',
 '.Over Reving (Atego 2)',
 '.Over Reving New Actros test',
 '.Over Revving (Atego 1)',
 '.Over Revving (LDV/Bus)',
 '.Over Speeding > 85km/h Warning (Act/Axo)',
 '.Over Speeding >100km/h Warning (Bus)',
 '.Over Speeding >105km/h (LG Fuso)',
 '.Over Speeding >110km/h (LG Fuso)',
 '.Over Speeding >130km/h Warning (LDV)',
 '.Over Speeding >83km/h Warning (Atego 1&2)',
 '.Over Speeding >95km/h (LG Fuso Buzzer)',
 '.Over rev warning (Act/Axo)',
 '.Over rev warning (Atego 1)',
 '.Over rev warning (Atego 2)',
 '.Over rev warning (LDV/Bus)',
 '.Over rev warning New Actros test',
 ".TAR - Over Speeding > 125km/h > 5sec (LDV's)",
 ".TAR - Over Speeding > 135km/h (LDV's)",
 '.TAR - Over Speeding > 88km/h',
 '.TAR - Over Speeding > 90km/h',
 '.TAR - Over Speeding > 90km/h > 1 Min (Trucks)',
 '.TAR - Over Speeding > 95km/h',
 '.TAR - Over Speeding >105km/h >1min',
                                      '.Impact Detected >13 (MV)',
 '.TAR - Over Speeding >120km/h',
                                      'Over Speeding (Reports)',
 'Over revving',
 'Over revving (Reports)',
 'Over revving - WARNING',
 'Over speeding',
 'Over speeding - TIERED',
 'Over speeding - WARNING',
 'Over speeding in location',
 'Over speeding in location - EXCESSIVE DURATION',
 'Over speeding in location - EXCESSIVE SPEED',]

In [42]:
neutral_behaviour=set(behaviour)-set(negative_behaviour)

In [43]:
neutral_behaviour

{'(Do not use) Ignition On >1km (MV)',
 '* Assistance Request',
 '* Ignition Off',
 '* Ignition On',
 '* Input 1 High',
 '* Input 1 Low',
 '* Input 2 Low',
 '* Low Battery',
 '* Low Internal Battery',
 '* Low Remote Control Battery',
 '* Low Vehicle Battery',
 '* OBC unit reset',
 '* Panic Alert',
 '* Power Disconnected',
 '* Power Reconnected',
 '*After Hour',
 '*Asset in Workshop Hours',
 '*Left Depot Late 7-10',
 '.Actros Engine Brake',
 '.Actros foot brake > 50km/h',
 '.Alarm - Dash Tamper (2310I)',
 '.Alarm - Dash Tamper (Limp)',
 '.Alarm - Dash Tamper (TL)',
 '.Alarm - Panic Pressed',
 '.Alarm - Panic Pressed (MV)',
 '.Alarm - Power Down',
 '.Alert - Stolen key in use (Tail lift)',
 '.Atego Back Door Open magnetic switch',
 '.Back Door Open',
 '.Cargo Door Closed',
 '.Cargo Door Open',
 '.Driving > 5hrs / 400km with no rest break',
 '.Driving @11am (MV)',
 '.Driving @8pm (MV)',
 '.GSM Jamming Detected (Limp)',
 '.ID Key Relay Switch (Tail lift de-activation)',
 '.ID Key Removed (

In [44]:
events_df[events_df.Description.isin(['*No Go Zone','* Impact Detect Moderate',
 '* Impact Detect Severe','* Harsh Cornering',
 '* Harsh Left Cornering',
 '* Harsh Right Cornering','.TAR - Over Speeding > 100km/h (Trucks)',
                                      '.Alert - Excessive Over Revving (>2400 RPM)',
 '.Alert - Over Speeding >100km/h (Trucks)',
 '.Alert - Over Speeding >120km/h (Bus)',
 '.Alert - Over Speeding >140km/h (LDV)',
 '.Alert - Possible Accident > 20km/h/s',
 '.Alert - Prohibited driving (22h00 - 02h45)',
'.Harsh Braking > 10km/h/s (MV)(LDV)',
 '.Harsh Braking > 8km/h/s (MV)(Trucks)',
 '.Harsh Braking >10km/h/s (LDV/Fuso F) (MV)',
                                      '.Over Reving (Act/Axo)',
 '.Over Reving (Atego 2)',
 '.Over Reving New Actros test',
 '.Over Revving (Atego 1)',
 '.Over Revving (LDV/Bus)',
 '.Over Speeding > 85km/h Warning (Act/Axo)',
 '.Over Speeding >100km/h Warning (Bus)',
 '.Over Speeding >105km/h (LG Fuso)',
 '.Over Speeding >110km/h (LG Fuso)',
 '.Over Speeding >130km/h Warning (LDV)',
 '.Over Speeding >83km/h Warning (Atego 1&2)',
 '.Over Speeding >95km/h (LG Fuso Buzzer)',
 '.Over rev warning (Act/Axo)',
 '.Over rev warning (Atego 1)',
 '.Over rev warning (Atego 2)',
 '.Over rev warning (LDV/Bus)',
 '.Over rev warning New Actros test',
 ".TAR - Over Speeding > 125km/h > 5sec (LDV's)",
 ".TAR - Over Speeding > 135km/h (LDV's)",
 '.TAR - Over Speeding > 88km/h',
 '.TAR - Over Speeding > 90km/h',
 '.TAR - Over Speeding > 90km/h > 1 Min (Trucks)',
 '.TAR - Over Speeding > 95km/h',
 '.TAR - Over Speeding >105km/h >1min',
                                      '.Impact Detected >13 (MV)',
 '.TAR - Over Speeding >120km/h',
                                      'Over Speeding (Reports)',
 'Over revving',
 'Over revving (Reports)',
 'Over revving - WARNING',
 'Over speeding',
 'Over speeding - TIERED',
 'Over speeding - WARNING',
 'Over speeding in location',
 'Over speeding in location - EXCESSIVE DURATION',
 'Over speeding in location - EXCESSIVE SPEED',])].shape

(67674, 20)

In [45]:
events_df[events_df.Description=='.TAR - Alert Possible Accident >20km/h/s'].shape

(4, 20)

In [46]:
events_df_grouped = events_df.groupby(['DriverId_u0', 'Description'])

In [47]:
events_df_grouped_count=events_df_grouped.aggregate(np.size)

In [48]:
events_df_grouped_count.reset_index(inplace=True)

In [49]:
events_df_grouped_count

,DriverId_u0,Description,Name,EventType,__KEY_root,Armed,AssetId_u0,EventDateTime,EventId,EventLatitude,EventLongitude,EventSpeed,EventTotalTime,EventTypeId,GroupId,OdometerKilometres_u0,OdometerKilometresEnd_u0,Priority,Source,Value
0,-9.174768e+18,*After Hour,4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
1,-9.174768e+18,.TAR - Excessive Idling > 10min,4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
2,-9.174768e+18,GSM Status - Possible Jamming,100,100,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0,100,100,100.0,100.0,100.0,100,100.0
3,-9.174768e+18,Idle,204,204,204.0,204.0,204.0,204,204.0,204.0,204.0,204.0,204.0,204,204,204.0,204.0,204.0,204,204.0
4,-9.174768e+18,Over speeding,126,126,126.0,126.0,126.0,126,126.0,126.0,126.0,126.0,126.0,126,126,126.0,126.0,126.0,126,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,9.218301e+18,Over Speeding (Reports),4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
1834,9.218301e+18,Road Speed Overspeeding,20,20,20.0,20.0,20.0,20,20.0,20.0,20.0,20.0,20.0,20,20,20.0,20.0,20.0,20,20.0
1835,9.220211e+18,Idle - excessive,2,2,2.0,2.0,2.0,2,2.0,2.0,2.0,2.0,2.0,2,2,2.0,2.0,2.0,2,2.0
1836,9.220211e+18,Over revving,36,36,36.0,36.0,36.0,36,36.0,36.0,36.0,36.0,36.0,36,36,36.0,36.0,36.0,36,36.0


The following creates a pivot table of the number of occurences of a particular event type:

In [52]:
events_df_grouped_pivot=events_df_grouped_count.pivot_table(index='DriverId_u0', columns='Description', values='GroupId',fill_value=0)

In [53]:
events_df_grouped_pivot

Description,*After Hour,*Asset in Workshop Hours,.Harsh Braking > 8km/h/s (MV)(Trucks),.Harsh Braking >10km/h/s (LDV/Fuso F) (MV),.Impact Detected >13 (MV),.TAR - Alert Possible Accident >20km/h/s,.TAR - Excessive Idling > 10min,.TAR - Over Speeding > 100km/h (Trucks),Battery disconnection,GPS Jamming Detected - Critical,...,Harsh braking,Harsh braking (Reports),Idle,Idle - excessive,Ignition On [Activate Relay Drive Off),Over Speeding (Reports),Over revving,Over revving (Reports),Over speeding,Road Speed Overspeeding
DriverId_u0,,,,,,,,,,,,,,,,,,,,,
-9.174768e+18,4,0,0,0,0,0,4,0,0,0,...,0,0,204,0,0,0,0,0,126,156
-9.145267e+18,0,0,4,0,0,0,0,0,0,0,...,0,2,0,0,0,82,0,0,0,18
-9.124200e+18,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,10,0,0,0,90
-9.107084e+18,0,0,8,0,0,0,0,0,0,0,...,0,2,0,0,0,6,0,0,0,8
-9.101669e+18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,66,0,0,0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.132976e+18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,12,0,0,0,14
9.147461e+18,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,170
9.153934e+18,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,82,0,0,0,34


In [57]:
events_df.head()


,Name,Description,EventType,__KEY_root,Armed,AssetId_u0,DriverId_u0,EventDateTime,EventId,EventLatitude,EventLongitude,EventSpeed,EventTotalTime,EventTypeId,GroupId,OdometerKilometres_u0,OdometerKilometresEnd_u0,Priority,Source,Value
0,Bakers Transport (R),.Harsh Braking > 8km/h/s (MV)(Trucks),Custom,NaN,NaN,-3.089065e+17,-6.169690e+18,2020-09-10T06:23:33,2.804730e+18,-32.962387,27.919195,21.0,0.0,-6962998577837367602,-7124170799261977621,791620.3,NaN,NaN,ENT,9.0
1,Bakers Transport (R),.Harsh Braking > 8km/h/s (MV)(Trucks),Custom,NaN,NaN,-3.089065e+17,-6.169690e+18,2020-09-10T06:23:34,2.804730e+18,-32.962391,27.919189,6.0,2.0,-6962998577837367602,-7124170799261977621,791620.3,NaN,NaN,ENT,11.0
2,Bakers Transport (R),.Harsh Braking > 8km/h/s (MV)(Trucks),Custom,NaN,NaN,-3.089065e+17,1.819075e+18,2020-09-09T10:47:34,2.804427e+18,-31.868444,26.931334,9.0,0.0,-6962998577837367602,-7124170799261977621,791263.6,NaN,NaN,ENT,9.0
3,Bakers Transport (R),.Harsh Braking > 8km/h/s (MV)(Trucks),Custom,NaN,NaN,-3.089065e+17,1.819075e+18,2020-09-09T10:47:35,2.804427e+18,-31.868441,26.931330,2.0,0.0,-6962998577837367602,-7124170799261977621,791263.6,NaN,NaN,ENT,9.0
4,Bakers Transport (R),.Harsh Braking > 8km/h/s (MV)(Trucks),Custom,NaN,NaN,-3.089065e+17,1.819075e+18,2020-09-09T11:15:57,2.804434e+18,-31.717388,27.199917,21.0,0.0,-6962998577837367602,-7124170799261977621,791299.6,NaN,NaN,ENT,9.0


Look at duration of each event by duration:

In [58]:
pd.pivot_table(events_df, values='EventTotalTime', 
                    index=['DriverId_u0'], 
                    columns='Description',
                    aggfunc='sum')

Description,*After Hour,*Asset in Workshop Hours,.Harsh Braking > 8km/h/s (MV)(Trucks),.Harsh Braking >10km/h/s (LDV/Fuso F) (MV),.Impact Detected >13 (MV),.TAR - Alert Possible Accident >20km/h/s,.TAR - Excessive Idling > 10min,.TAR - Over Speeding > 100km/h (Trucks),Battery disconnection,GPS Jamming Detected - Critical,...,Harsh braking,Harsh braking (Reports),Idle,Idle - excessive,Ignition On [Activate Relay Drive Off),Over Speeding (Reports),Over revving,Over revving (Reports),Over speeding,Road Speed Overspeeding
DriverId_u0,,,,,,,,,,,,,,,,,,,,,
-9.174768e+18,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,7088.0,NaN,NaN,NaN,NaN,NaN,852.0,7804.0
-9.145267e+18,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,228.0,NaN,NaN,NaN,6.0
-9.124200e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,18.0,NaN,NaN,NaN,244.0
-9.107084e+18,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,182.0,NaN,NaN,NaN,0.0
-9.101669e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1192.0,NaN,NaN,NaN,196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.132976e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,52.0
9.147461e+18,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,3306.0
9.153934e+18,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,556.0,NaN,NaN,NaN,286.0


In [ ]:
X=events_df_grouped_pivot.to_numpy()

In [ ]:
X.shape

In [ ]:
list(events_df_grouped_pivot).index('.Harsh Braking > 8km/h/s (MV)(Trucks)')

In [ ]:
list(events_df_grouped_pivot).index('Road Speed Overspeeding')

In [ ]:
list(events_df_grouped_pivot)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [ ]:
list(events_df_grouped_pivot)

In [ ]:
linked = linkage(X, 'single')

labelList = list(events_df_grouped_pivot)

plt.figure(figsize=(10, 7))
dendrogram(linked,
            orientation='top',
#             labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
cluster.fit_predict(X)

In [ ]:
import scipy.cluster.hierarchy as shc

plt.figure(figsize=(10, 7))
plt.title("Driver Dendograms")
dend = shc.dendrogram(shc.linkage(X, method='ward'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
cluster.fit_predict(X)

In [ ]:
print('Factor: ',list(events_df_grouped_pivot)[18],', and factor: ',list(events_df_grouped_pivot)[24])

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(X[:,18], X[:,24], c=cluster.labels_,cmap='rainbow')

### Assets

In [ ]:
assets_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/Asset Master.xlsx', sheet_name='Sheet1')

In [ ]:
list(assets_df)

In [ ]:
set(assets_df.AssetTypeId)

In [ ]:
len(set(assets_df.Model))

In [ ]:
(set(assets_df.Year.fillna(0)))

### Trips

In [ ]:
trips_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/Trips.xlsx', sheet_name='Sheet1')

In [ ]:
list(trips_df)

In [ ]:
trips_df.shape

Drivers had multiple trips:

In [ ]:
len(set(trips_df.DriverId))

In [ ]:
# Are trips unique:
len(set(trips_df.TripId)) # They are!

In [ ]:
len(set(trips_df.AssetId.fillna(0)))

In [ ]:
events_df_grouped = trips_df.groupby(['DriverId', 'Description'])

In [ ]:
trips_df.head()

In [ ]:
set(trips_df.StartPositionId)

In [ ]:
trips_df.TripStart